<a href="https://colab.research.google.com/github/Talizg03/weather/blob/main/29_1_25_project_remove_half_of_the_rows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

df = pd.read_csv("Train.csv")
#print(df.head())
df.head()

Saving Train.csv to Train.csv


<ipython-input-1-1280e8b6c072>:5: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Train.csv")


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   401125 non-null  int64  
 1   SalePrice                 401125 non-null  int64  
 2   MachineID                 401125 non-null  int64  
 3   ModelID                   401125 non-null  int64  
 4   datasource                401125 non-null  int64  
 5   auctioneerID              380989 non-null  float64
 6   YearMade                  401125 non-null  int64  
 7   MachineHoursCurrentMeter  142765 non-null  float64
 8   UsageBand                 69639 non-null   object 
 9   saledate                  401125 non-null  object 
 10  fiModelDesc               401125 non-null  object 
 11  fiBaseModel               401125 non-null  object 
 12  fiSecondaryDesc           263934 non-null  object 
 13  fiModelSeries             56908 non-null   o

remove col: 45  Thumb                     0 non-null      object
 46  Pattern_Changer            
 47  Grouser_Type             
 48  Backhoe_Mounting       
 49  Blade_Type          
 50  Travel_Controls     
 51  Differential_Type   
 52  Steering_Controls

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance


X, y = df.drop(columns='SalePrice'), df['SalePrice']
feature_names = X.columns

def train_model(X, y):
    for col in X.columns:
        if X[col].dtype == 'object':
            X[col] = X[col].astype('category')

        if X[col].dtype == 'category':
            X[col] = X[col].cat.codes

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the Random Forest model
    rf = RandomForestRegressor(random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)

    # Evaluate model performance using RMSE
    from sklearn.metrics import mean_squared_error
    y_pred = rf.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    y_train_pred = rf.predict(X_train)
    train_rmse = mean_squared_error(y_train, y_train_pred) ** 0.5
    print("RMSE Baseline accuracy:", y_test.std())
    print(f"Train: Root Mean Squared Error (RMSE): {train_rmse}")
    print(f"Test: Root Mean Squared Error (RMSE): {rmse}")
    return rf, (X_train, X_test, y_train, y_test)

rf, xs = train_model(X, y)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Function to preprocess and convert categorical columns to category codes
def train_model(X, y):
    for col in X.columns:
        if X[col].dtype == 'object':
            X[col] = X[col].astype('category')  # Convert object columns to category type

        if X[col].dtype == 'category':
            X[col] = X[col].cat.codes  # Convert category columns to numerical codes

    return X, y

# Randomly sample half of the rows from the dataset
df_half_random = df.sample(frac=0.5, random_state=42)

# Separate features (X) and target (y) from the sampled DataFrame
X_half_random = df_half_random.drop(columns='SalePrice')  # Replace 'price' with your target column
y_half_random = df_half_random['SalePrice']  # Replace 'price' with your target column

# Preprocess the features (convert categorical columns to category codes)
X_half_random, y_half_random = train_model(X_half_random, y_half_random)

# Split the sampled half into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_half_random, y_half_random, test_size=0.2, random_state=42)

# Train the RandomForestRegressor model on the processed data
model = RandomForestRegressor(random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Now, you can make predictions, evaluate the model, etc.
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
# Evaluate model performance using RMSE
from sklearn.metrics import mean_squared_error
y_pred = rf.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
y_train_pred = rf.predict(X_train)
train_rmse = mean_squared_error(y_train, y_train_pred) ** 0.5
print("RMSE Baseline accuracy:", y_test.std())
print(f"Train: Root Mean Squared Error (RMSE): {train_rmse}")
print(f"Test: Root Mean Squared Error (RMSE): {rmse}")
#return model (X_train, X_test, y_train, y_test)

# Evaluate model performance using RMSE
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
y_train_pred = model.predict(X_train)
train_rmse = mean_squared_error(y_train, y_train_pred) ** 0.5

# Evaluate the model (example: calculating the R^2 score)
print("Training R^2 Score:", model.score(X_train, y_train))
print("Testing R^2 Score:", model.score(X_test, y_test))


RMSE Baseline accuracy: 22878.88247012821
Train: Root Mean Squared Error (RMSE): 15527.879147540965
Test: Root Mean Squared Error (RMSE): 15399.958591538154
Training R^2 Score: 0.9743287262239543
Testing R^2 Score: 0.8189236302908602


# Summary of the Results:



**Baseline RMSE**: The RMSE for a baseline model, which predicts the mean value for all observations (22,878.88). This is a reference point for the performance of your model.
**Train RMSE: **The model's RMSE on the training data (15,527.88). This shows how well the model fits the training data.
**Test RMSE:** The model's RMSE on the test data (15,399.96). This shows how well the model generalizes to new, unseen data.
**Training R²:** 0.9743 – The model is performing extremely well on the training data.
**Testing R²:** 0.8189 – The model performs well on the test data, though there is a slight drop from the training performance. This suggests that while the model generalizes well to unseen data, there might be some overfitting, or the test data has slightly different characteristics from the training data.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Function to preprocess and convert categorical columns to category codes
def train_model(X, y):
    # Preprocessing: Convert object columns to category type and category columns to numerical codes
    for col in X.columns:
        if X[col].dtype == 'object':
            X[col] = X[col].astype('category')  # Convert object columns to category type

        if X[col].dtype == 'category':
            X[col] = X[col].cat.codes  # Convert category columns to numerical codes

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the RandomForestRegressor model on the processed data
    rf = RandomForestRegressor(random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)

    # Evaluate model performance using RMSE
    y_pred = rf.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    y_train_pred = rf.predict(X_train)
    train_rmse = mean_squared_error(y_train, y_train_pred) ** 0.5

    # Print the RMSE results
    print("RMSE Baseline accuracy:", y_test.std())
    print(f"Train: Root Mean Squared Error (RMSE): {train_rmse}")
    print(f"Test: Root Mean Squared Error (RMSE): {rmse}")

    # Return the trained model and the data used
    return rf, (X_train, X_test, y_train, y_test)

# Example: Load your DataFrame (make sure 'price' is the target column and X contains features)
df_half_random = df.sample(frac=0.5, random_state=42)

# Separate features (X) and target (y) from the sampled DataFrame
X_half_random = df_half_random.drop(columns='SalePrice')  # Replace 'price' with your target column
y_half_random = df_half_random['SalePrice']  # Replace 'price' with your target column

# Call the function to train the model
rf, xs = train_model(X_half_random, y_half_random)


RMSE Baseline accuracy: 22878.88247012821
Train: Root Mean Squared Error (RMSE): 3693.2710078715086
Test: Root Mean Squared Error (RMSE): 9735.545332576477


**Summary of the Results:**
Baseline RMSE (22,878.88): A simple model that predicts the mean value for all observations would have an RMSE of 22,878.88.

**Training RMSE (3,693.27)**: The model's RMSE on the training dataset is 3,693.27, which shows that the model is fitting the training data well and making good predictions for the data it has seen.

**Testing RMSE (9,735.55):** The model's RMSE on the test dataset is 9,735.55, which indicates that the model is still making good predictions on new, unseen data, though the error is slightly higher than on the training set. This is expected, as it is typical for a model to perform a little worse on unseen data compared to the training data.

What Does This Mean?:
Model Performance: The training RMSE is much lower than the baseline RMSE, which means your model is significantly better than predicting the mean value of the target. It is able to learn patterns in the data and make predictions with considerably less error.

Generalization: The test RMSE is higher than the train RMSE, which is a typical scenario in machine learning. This could be due to overfitting (where the model performs exceptionally well on the training data but is not generalizing as well on the test data). However, the test RMSE is still much lower than the baseline, which means the model is still generalizing well to unseen data.

In [10]:
from sklearn.inspection import permutation_importance
def perm_importance_df(rf, xs):
    X_train, X_test, y_train, y_test = xs

    # Compute Permutation Importance
    perm_importance = permutation_importance(rf, X_test, y_test, n_repeats=5, random_state=42, n_jobs=-1)

    # Display the results
    importance_df = pd.DataFrame({
        "Feature": rf.feature_names_in_,
        "Importance": perm_importance.importances_mean,
        "Std Deviation": perm_importance.importances_std,
        'model importance': rf.feature_importances_
    }).sort_values(by="Importance", ascending=False)

    display(importance_df)
    return importance_df

perm_importance_df(rf, xs)

PicklingError: Could not pickle the task to send it to the workers.